# AIDA 2154 – Final Project: Deep Learning-based Monitoring of Solar Panels

**Group 1:** Mark, Herve, Kelsey  
**Course:** Computer Vision (Fall 2025)  
**Instructor:** Dr. Muhammad Tufail

## Project Overview
This notebook implements an **Object Detection** system using **YOLOv11** to identify thermal anomalies in photovoltaic modules. The system is trained on the *Infrared Solar Modules* dataset to detect and localize 12 distinct classes of faults (e.g., Hotspots, Diode failures) and environmental issues (Vegetation, Soiling).

### Methodology
1.  **Data Preparation:** Dynamic configuration of the dataset paths.
2.  **Model Training:** Fine-tuning a pre-trained `yolo11n` (Nano) model.
3.  **Evaluation:** Analyzing Mean Average Precision (mAP) and confusion matrices.
4.  **Inference:** Demonstrating the model on unseen test images.

In [ ]:
# Cell 1: Environment Setup & GPU Check
# Installs dependencies if missing, and imports necessary libraries.

import os
import sys
import yaml
import random
import glob
import cv2
import torch
import matplotlib.pyplot as plt

try:
    import ultralytics
    from ultralytics import YOLO
except ImportError:
    print("Installing Ultralytics YOLO...")
    !pip install -q ultralytics
    import ultralytics
    from ultralytics import YOLO

# Verify installation
ultralytics.checks()

# HARDWARE CHECK
if torch.cuda.is_available():
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
    BATCH_SIZE = 16  # Standard Production Batch Size
else:
    print("⚠️ GPU NOT Detected. Running on CPU.")
    BATCH_SIZE = 4

print(f"Project Root: {os.getcwd()}")

Ultralytics 8.3.235  Python-3.11.14 torch-2.10.0.dev20251205+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
Setup complete  (20 CPUs, 31.1 GB RAM, 362.6/927.8 GB disk)
✅ GPU Detected: NVIDIA GeForce RTX 5070 Laptop GPU
Project Root: C:\Users\markm\Desktop\AIDA2154_Solar_Panel_Project


In [ ]:
# Cell 2: Dynamic Data Configuration
# This cell automatically locates the dataset and creates the YOLO config file.
# This ensures the notebook runs on any machine without manual path editing.

# 1. Locate the Dataset
possible_locations = [
    os.path.join("InfraredSolarModules", "ImageSet"),  # Expected location
    "datasets/InfraredSolarModules/ImageSet",        # Alternative
    "ImageSet"                                       # Flat structure
]

dataset_root = None
for path in possible_locations:
    if os.path.exists(os.path.abspath(path)):
        dataset_root = os.path.abspath(path)
        break

if dataset_root is None:
    raise FileNotFoundError("❌ CRITICAL: Dataset not found. Please ensure'InfraredSolarModules/ImageSet' exists.")

print(f"✅ Dataset found at: {dataset_root}")

# 2. Create the Configuration Dictionary
config = {
    'path': dataset_root,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': {
        0: 'Cell',
        1: 'Cell-Multi',
        2: 'Cracking',
        3: 'Diode',
        4: 'Diode-Multi',
        5: 'Hot-Spot',
        6: 'Hot-Spot-Multi',
        7: 'No-Anomaly',
        8: 'Offline-Module',
        9: 'Shadowing',
        10: 'Soiling',
        11: 'Vegetation'
    }
}

# 3. Save as YAML
config_filename = 'final_solar_config.yaml'
with open(config_filename, 'w') as f:
    yaml.dump(config, f)

print(f"✅ Config file '{config_filename}' generated.")

✅ Dataset found at: C:\Users\markm\Desktop\AIDA2154_Solar_Panel_Project\InfraredSolarModules\ImageSet
✅ Config file 'final_solar_config.yaml' generated.


In [ ]:
# Cell 3: Model Training (Task 3)
# We use YOLOv11 Nano (Detection) for speed and efficiency.

# Load Model
model = YOLO('yolo11n.pt')

print(f"🚀 Starting Training Pipeline with Batch Size: {BATCH_SIZE}...")

# Train
results = model.train(
    data='final_solar_config.yaml',
    epochs=15,
    imgsz=640,          # Standard detection size
    batch=BATCH_SIZE,   # Set to 16 (Standard)
    project='Solar_Final_Runs',
    name='yolo11_detect',
    exist_ok=True,
    plots=True,

    # --- COMPATIBILITY FLAGS ---
    # These settings prevent kernel crashes on Windows/Nightly builds
    workers=0,           # Forces main-process loading (Prevents DataLoader crashes)
    amp=False,           # Disables Mixed Precision (Prevents SegFaults on new GPUs)
    cache=False,         # Reads from disk to avoid memory spikes
)

🚀 Starting Training Pipeline with Batch Size: 16...
Ultralytics 8.3.235  Python-3.11.14 torch-2.10.0.dev20251205+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=final_solar_config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11_detect, nbs=64, nms=False, opset=None, optimize

In [ ]:
# Cell 4: Validation & Metrics (Task 4)
# Evaluate the model on the validation set.

print("📊 Running Validation...")
metrics = model.val()

print(f"\n🏆 Results:")
print(f"   mAP@50:    {metrics.box.map50:.3f}")
print(f"   mAP@50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.mean_results()[0]:.3f}")
print(f"   Recall:    {metrics.box.mean_results()[1]:.3f}")

📊 Running Validation...
Ultralytics 8.3.235  Python-3.11.14 torch-2.10.0.dev20251205+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
YOLO11n summary (fused): 100 layers, 2,584,492 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 370.459.6 MB/s, size: 49.5 KB)
val: Scanning C:\Users\markm\Desktop\AIDA2154_Solar_Panel_Project\InfraredSolarModules\ImageSet\valid\labels.cache... 400 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 400/400 200.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 8.3it/s 3.0s<0.2s
                   all        400       1119      0.647      0.696      0.713      0.552
                  Cell         29         34      0.826      0.698      0.838      0.637
            Cell-Multi         39         77      0.461      0.506      0.504      0.383
              Cracking        138        209      0.577      0.752      0.725      0.591
        

In [ ]:
import os

# 1. Define the exact path where YOLO saved your results
# Based on your logs, this is: C:\Users\markm\Desktop\AIDA2154_Solar_Panel_Project\Solar_Final_Runs\yolo11_detect
project_dir = os.getcwd()
run_folder = os.path.join(project_dir, "Solar_Final_Runs", "yolo11_detect")

print("✅ PROJECT STATUS: COMPLETE")
print("==================================================")

# 2. Verify the folder exists
if os.path.exists(run_folder):
    print(f"📊 Your Training Results (Charts & Images) are saved here:")
    print(f"   📂 {run_folder}")
    print("\n   Copy and paste the line above into your File Explorer address bar.")
    print("   You will find:")
    print("     - results.png (Loss curves)")
    print("     - confusion_matrix.png")
    print("     - val_batch0_pred.jpg (Example detections)")
    print("     - weights/best.pt (Your trained model)")
else:
    print(f"❌ Error: Could not find results folder at: {run_folder}")
    print("   Did you change the 'project' or 'name' arguments in Cell 3?")

print("==================================================")

✅ PROJECT STATUS: COMPLETE
📊 Your Training Results (Charts & Images) are saved here:
   📂 C:\Users\markm\Desktop\AIDA2154_Solar_Panel_Project\Solar_Final_Runs\yolo11_detect

   Copy and paste the line above into your File Explorer address bar.
   You will find:
     - results.png (Loss curves)
     - confusion_matrix.png
     - val_batch0_pred.jpg (Example detections)
     - weights/best.pt (Your trained model)


In [ ]:
#Load the results so you don't have to run it again, but you can generate the reports again.

import os
import random
import glob
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# 1. Define the path to your SAVED model
# This path comes from your training log: "Results saved to C:\...\Solar_Final_Runs\yolo11_detect"
# We add "\weights\best.pt" to point to the actual model file.
saved_model_path = os.path.join("Solar_Final_Runs", "yolo11_detect", "weights", "best.pt")

if os.path.exists(saved_model_path):
    print(f"✅ Found trained model at: {saved_model_path}")

    # Load the model
    model = YOLO(saved_model_path)

    # 2. Run Inference on Test Images (Just like before)
    # Re-define the dataset root if the variable was lost
    dataset_root = os.path.abspath(os.path.join("InfraredSolarModules", "ImageSet"))
    test_images = glob.glob(os.path.join(dataset_root, "test", "images", "*.jpg"))

    if test_images:
        samples = random.sample(test_images, min(len(test_images), 3))
        print(f"🔍 Running inference on {len(samples)} images...")

        for img_path in samples:
            results = model.predict(img_path, conf=0.25, verbose=False)
            result_array = results[0].plot()
            result_rgb = cv2.cvtColor(result_array, cv2.COLOR_BGR2RGB)

            plt.figure(figsize=(10, 8))
            plt.imshow(result_rgb)
            plt.axis('off')
            plt.title(f"Detection: {os.path.basename(img_path)}")
            plt.show()
    else:
        print("⚠️ Model found, but could not find test images to display.")
else:
    print(f"❌ Could not find model at: {saved_model_path}")
    print("   Did you change the project name or path?")